## Laboratory 2
### Aaron Sigman

There are more satellites and data sources available to you. Using your Lab 2 notebook and the one we showed how to create your own notebook (attached) create a notebook adding the following satellites in natural and read false color display (both).

In [1]:
from IPython.display import Image
import ee
ee.Initialize()

In [2]:
import geemap

### Define a point

In [3]:
# Define point at Logan, UT
Lat=41.735210
Long=-111.834862
Point = ee.Geometry.Point([Long, Lat])

### Landsat8 Imagery for comparison

In [4]:
# Call in landsat imagery, selecting region, date, and sorting by cloud cover
landsat8 =ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(Point).filterDate('2021-06-01', '2021-09-01').sort('CLOUD_COVER_LAND').first()
# Select bands
image = landsat8.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])
image = image.multiply(0.0000275).add(-0.2)

In [5]:
# Call map, and add layers with bands you want
Map = geemap.Map() # from ipygee
bounds = Point.buffer(10000)
Map.centerObject(bounds)
trueColor = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.4}
falseColor= {'bands': ['SR_B5', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.4}
Map.addLayer(image, trueColor, name='Landsat8 True')
Map.addLayer(image, falseColor, name='Landsat8 False')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
# Get ASTER images for the study period and region of interest.
ASTERImages = ee.ImageCollection("ASTER/AST_L1T_003").filterDate('2019-04-01', '2019-10-31').filterBounds(Point)
# Mosaic adjacent images into a single image.
ASTERImage = ASTERImages.mosaic();

In [7]:
# ASTER doesn't have blue
IRBands=  ['B3N', 'B02', 'B01']
# Add layer to map
Map.addLayer(ASTERImage, {'bands':IRBands,'min': 0, 'max': 255}, name='ASTER Image')
Map

Map(bottom=195442.0, center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['positio…

In [8]:
# // Get SEN2 images for the study period and region of interest.
SEN2Images = ee.ImageCollection("COPERNICUS/S2").filterDate('2019-04-01', '2019-10-31').filterBounds(Point)
# // Mosaic adjacent images into a single image.
SEN2Image = SEN2Images.mosaic();
SEN2Image = SEN2Image.multiply(0.0001)

In [9]:
# Select bands
IRBands=  ['B8', 'B4', 'B3']
TrueBands=['B4','B3','B2']
# Add layers to map
Map.addLayer(SEN2Image, {'bands':IRBands,'min': 0, 'max': 1}, name='SEN2 NIR')
Map.addLayer(SEN2Image, {'bands':TrueBands,'min': 0, 'max': 1}, name='SEN2 True')
Map

Map(bottom=195442.0, center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['positio…

In [10]:
# Get SEN imagery
SEN3 = ee.ImageCollection("COPERNICUS/S3/OLCI").filterDate('2019-04-01', '2019-10-31').filterBounds(Point)
# Correct bands for band scales
rgb = SEN3.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance']).median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
NIR = SEN3.select(['Oa14_radiance','Oa08_radiance', 'Oa06_radiance']).median().multiply(ee.Image([0.0086512, 0.00876539, 0.0123538]))

visParams = {'min': 0,'max': 4,'gamma': 1.5}

In [11]:
Map.addLayer(rgb, visParams, name='SEN3 True')
Map.addLayer(NIR, visParams, name='SEN3 NIR')

Map

Map(bottom=195442.0, center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['positio…

In [12]:
# Get PROBA images
PROBA = ee.ImageCollection("VITO/PROBAV/C1/S1_TOC_100M").filterDate('2019-05-25', '2019-10-27').filterBounds(Point).sort("CLOUD_COVER_LAND").first()

In [13]:
# No green band, so combo here is NIR, R, B
IRBands=  ['NIR', 'RED', 'BLUE']
Map.addLayer(PROBA, {'bands':IRBands,'min': 20, 'max': 2000}, name='PROBA IR')
Map

Map(bottom=195442.0, center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['positio…

In [14]:
VIIRSImages = ee.ImageCollection("NOAA/VIIRS/001/VNP09GA").filterDate('2019-05-25', '2019-10-27').filterBounds(Point).sort("CLOUD_COVER_LAND").first()
VIIRSImage = VIIRSImages.multiply(0.0001)

['I1', 'I2', 'I3', 'M1', 'M2', 'M3', 'M4', 'M5', 'M7', 'M8', 'M10', 'M11', 'SensorAzimuth', 'SensorZenith', 'SolarAzimuth', 'SolarZenith', 'iobs_res', 'num_observations_1km', 'num_observations_500m', 'obscov_1km', 'obscov_500m', 'orbit_pnt', 'QF1', 'QF2', 'QF3', 'QF4', 'QF5', 'QF6', 'QF7']


In [15]:
IRBands=  ['M7', 'M5', 'M4']
TrueBands=['M5','M4','M3']
Map.addLayer(VIIRSImage, {'bands':IRBands,'min': 0, 'max': 1}, name='VIIRS IR')
Map.addLayer(VIIRSImage, {'bands':TrueBands,'min': 0, 'max': 1}, name='VIIRS True')

Map

Map(bottom=195442.0, center=[41.7352272667938, -111.83465310180142], controls=(WidgetControl(options=['positio…

In [16]:
HYPImages = ee.ImageCollection("EO1/HYPERION").filterDate('2001-05-01', '2002-5-01').filterBounds(Point)
HYPImage = HYPImages.mosaic()

In [17]:
# NIR, R, G
IRBands=  ['B050', 'B023', 'B015']
TrueBands=['B023','B015','B013']
Map.addLayer(HYPImage, {'bands':IRBands,'min': 0, 'max': 7000}, name='HYPERION IR')
Map.addLayer(HYPImage, {'bands':TrueBands,'min': 0, 'max': 7000}, name='HYPERION True')
Map

Map(bottom=49086.0, center=[41.73545418490723, -111.83395385742189], controls=(WidgetControl(options=['positio…